<a href="https://colab.research.google.com/github/saotomryo/Use_MMDetection/blob/main/Use_MMDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import json
from skimage import measure
import numpy as np
print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device = ", device)

1.9.0+cu111
device =  cuda


## 環境準備 MMCV MMDetectionのインストール

In [ ]:
# MMCVのインストール（30分程度かかります）

!pip install mmcv-full==1.3.8

     |████████████████████████████████| 312 kB 2.8 MB/s 
     |████████████████████████████████| 185 kB 57.5 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.8-cp37-cp37m-linux_x86_64.whl size=29583804 sha256=a7f9f9a78eb0868f6adb458f6238ef1fc55951ded97d441d0d9957e33abb70b4
  Stored in directory: /root/.cache/pip/wheels/b1/c4/8d/b5bab7347e1b71280505e58eca95d00a53a524a7d5fa3ac89f
Successfully built mmcv-full


In [ ]:
HOME_PATH = "/content"

In [ ]:

!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -r requirements/build.txt
!pip install "git+https://github.com/open-mmlab/cocoapi.git#subdirectory=pycocotools"
!pip install -v -e .  # or "python setup.py develop"

Cloning into 'mmdetection'...
remote: Enumerating objects: 21231, done.
remote: Total 21231 (delta 0), reused 0 (delta 0), pack-reused 21231
Receiving objects: 100% (21231/21231), 24.89 MiB | 23.66 MiB/s, done.
Resolving deltas: 100% (14872/14872), done.
/content/mmdetection
  Cloning https://github.com/open-mmlab/cocoapi.git to /tmp/pip-req-build-59lo65lw
  Running command git clone -q https://github.com/open-mmlab/cocoapi.git /tmp/pip-req-build-59lo65lw
  Created wheel for mmpycocotools: filename=mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl size=264249 sha256=42555851d11c2df589ac3485cadb4e43d6acf5ac2d0692b189b6655baad2c8e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-na4zvbbp/wheels/d4/38/2b/ec4e44ec87a2b8692cd999bb33a831823b8071d0f89661297e
Successfully built mmpycocotools
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local

## コンフィグファイルの編集

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/content/mmdetection/configs/yolo/yolov3_d53_mstrain-608_273e_coco.py')


In [ ]:
# MMdetectionの分類結果は、COCODatasetで定義された80分類に従って、表示されます。
# 表示名を変更するには、以下のファイルの'cocodataset'の部分を編集してください。
# /content/mmdetection/mmdet/datasets/coco.py
# /content/mmdetection/mmdet/core/evaluation/class_names.py

# 編集後にすでに編集ずみファイルを持っている場合は以下のように内容をコピーして置き換えてください。


!cp "コピーもとファイル" "/content/mmdetection/mmdet/datasets/coco.py" -r
!cp "コピーもとファイル" "/content/mmdetection/mmdet/core/evaluation/class_names.py" -r 

In [ ]:
# コンフィグを変更する
# yolov3

data_root = '画像データのルートフォルダ'

cfg.model.bbox_head.num_classes = 4
#cfg.num_class = 80
cfg.data_root = data_root

# trainデータの定義
cfg.data.train.ann_file = '学習用のアノテーションファイル'
cfg.data.train.img_prefix = data_root + '学習用画像データを保存したフォルダ'

# valデータの定義
cfg.data.val.ann_file = '検証用のアノテーションファイル'
cfg.data.val.img_prefix = data_root + '検証用画像データを保存したフォルダ'

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


# 編集したコンフィグをファイル出力
cfg.dump('/content/yolo.py')


In [ ]:
!mkdir -p "$HOME_PATH"/work_train
!python tools/train.py \
    '/content/yolo.py' \
    --work-dir "$HOME_PATH"/work_train


## 学習結果の確認

In [ ]:
from argparse import ArgumentParser
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
model = init_detector("/content/yolo.py", "学習済みモデル", device=device)
result = inference_detector(model, "予測結果を確認する画像")
show_result_pyplot(model, "予測結果を確認する画像", result, score_thr=0.4)

## 予測タスクの実施

In [ ]:
from argparse import ArgumentParser
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
import os

testimage_path　 = "テスト画像をおいたフォルダ"

imgaes = glob(testimage_path)

# Specify the path to model config and checkpoint file
config_file = '/content/drive/MyDrive/Detection/mmdetection/configs/htc/htc_without_semantic_r50_fpn_1x_coco.py'
checkpoint_file = '/content/drive/MyDrive/Detection/work_train/epoch_18.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file,device=device)



bbox_dict1 = {}

for image in images:
    
    if os.path.exists(image):

        result = inference_detector(model, image)
        
        i = 0
        result_segmentation = []

        for i in range(6):
            result_segmentation1 = []
            tmp_bbox = []
            if len(result[0][i]):
                for j in range(len(result[0][i])):
                    if len(result_segmentation1) == 0:
                        contours = measure.find_contours(np.array(result[1][i][j]), 0.5)
                        result_segmentation1.append([contours[0].tolist(),result[0][i][j][0:5].tolist()])
                        tmp_bbox.append(result[0][i][j][0:4].tolist())

                    else:
                        if check_bbox(result[0][i][j][0:4].tolist(),tmp_bbox):
                            contours = measure.find_contours(np.array(result[1][i][j]), 0.5)
                            result_segmentation1.append([contours[0].tolist(),result[0][i][j][0:5].tolist()])
                            tmp_bbox.append(result[0][i][j][0:4].tolist())                       


                category = category_map[i+1]
                bbox_dict = {
                    category:result_segmentation1
                }

                if len(result_segmentation1):
                    result_segmentation.append(bbox_dict)
        
        bbox_dict1[img] = result_segmentation


with open('結果を出力するパス', 'w') as f:
    json.dump(bbox_dict1, f, ensure_ascii=False)



/content/drive/My Drive/Detection/mmdetection/mmdet/datasets/utils.py:68: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
